기상청의 기온 분석 csv 파일에서 평균기온 데이터를 이용하여 온도를 예측한다.

---
지점: 서울 <br>
자료구분: 일 <br>
기간: 2021-06-01 부터 2024-05-31

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
# 데이터 불러오기
data = pd.read_csv("/content/drive/MyDrive/전산물리_데이터/ta_20240613055533.csv", encoding="cp949")

In [ ]:
# 빈 데이터 확인하기, 삭제
print(data.isnull().sum())
data = data.dropna()

날짜         0
평균기온(℃)    1
최저기온(℃)    2
최고기온(℃)    1
dtype: int64


In [ ]:
# 빈 데이터 확인 -> 0
print(data.isnull().sum())

날짜         0
평균기온(℃)    0
최저기온(℃)    0
최고기온(℃)    0
dtype: int64


In [ ]:
# 열 이름 파악
print(data.columns)

Index(['날짜', '평균기온(℃)', '최저기온(℃)', '최고기온(℃)'], dtype='object')


In [ ]:
# 남길 열: '평균기온'
data = data.drop('날짜', axis=1)
data = data.drop('최저기온(℃)', axis=1)
data = data.drop('최고기온(℃)', axis=1)

In [ ]:
print(data)

      평균기온(℃)
0        20.2
1        23.2
2        19.1
3        18.5
4        20.8
...       ...
1091     18.5
1092     20.4
1093     21.2
1094     20.7
1095     20.4

[1095 rows x 1 columns]


In [ ]:
# 데이터 전처리(데이터 정규화)
data_max = data[['평균기온(℃)']].max()
data_min = data[['평균기온(℃)']].min()
scaled_data = (data[['평균기온(℃)']] - data_min) / (data_max - data_min)
scaled_data = scaled_data.values

In [ ]:
# 학습 데이터와 테스트 데이터로 분리
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [ ]:
# 시퀀스 데이터셋 생성 함수
def make_sequence_dataset(data, window_size):
    sequence_data = []
    sequence_label = []
    for i in range(len(data) - window_size -1):
        sequence_data.append(data[i:i+window_size])
        sequence_label.append(data[i+window_size])
    return np.array(sequence_data), np.array(sequence_label)

In [ ]:
# 시퀀스 길이 정의
window_size = 730

# 학습 데이터를 시퀀스 데이터셋으로 변환
train_data, train_label = make_sequence_dataset(train_data, window_size)

In [ ]:
# LSTM 모델 생성
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, input_shape=(train_data.shape[1], train_data.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer="adam",
              loss=tf.keras.losses.MeanSquaredError())

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 32)                4352      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4385 (17.13 KB)
Trainable params: 4385 (17.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 모델 학습
model.fit(train_data, train_label, epochs=5, batch_size=1)

Epoch 1/5
145/145 [==============================] - 25s 158ms/step - loss: 0.0534
Epoch 2/5
145/145 [==============================] - 21s 143ms/step - loss: 0.0112
Epoch 3/5
145/145 [==============================] - 20s 141ms/step - loss: 0.0113
Epoch 4/5
145/145 [==============================] - 23s 157ms/step - loss: 0.0112
Epoch 5/5
145/145 [==============================] - 21s 142ms/step - loss: 0.0098


In [ ]:
# 10일 뒤의 온도 예측하기
predictions = []
for i in range(10):
    last_window = scaled_data[-window_size:]
    prediction = model.predict(last_window[np.newaxis, ...])[0]
    predictions.append(prediction)
    scaled_data = np.concatenate([scaled_data[-window_size+1:], prediction[np.newaxis, ...]], axis=0)
predictions = pd.DataFrame(predictions, columns=['평균기온(℃)'])

# 데이터 역정규화
predictions = predictions * (data_max - data_min) + data_min
print(predictions)

1/1 [==============================] - 0s 43ms/step
     평균기온(℃)
0  22.311451
1  22.512520
2  22.742949
3  22.994801
4  23.257328
5  23.523362
6  23.788027
7  24.047982
8  24.300973
9  24.545490
